In [1]:
!wget https://www.encodeproject.org/report.tsv?type=Experiment&status=released

--2019-06-07 02:15:21--  https://www.encodeproject.org/report.tsv?type=Experiment
Resolving www.encodeproject.org (www.encodeproject.org)... 34.211.244.144
Connecting to www.encodeproject.org (www.encodeproject.org)|34.211.244.144|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/tsv]
Saving to: ‘report.tsv?type=Experiment’

report.tsv?type=Exp     [              <=>   ]  13.96M   280KB/s    in 50s     

2019-06-07 02:16:18 (285 KB/s) - ‘report.tsv?type=Experiment’ saved [14637719]



In [0]:
!mv report.tsv\?type\=Experiment metadata

In [0]:
!sed -i '1d' metadata
import pandas as pd

In [0]:
import pandas as pd
data = pd.read_csv("metadata", sep='\t')

In [5]:
# Format of dataset 
print(data.loc[0])
relevant_data = data[['Accession','Assay name','Assay title', 'Biosample term name', 'Target of assay', 'Organism', 'Biosample treatment']]
ids_of_data=data['Accession']

ID                                                                 /experiments/ENCSR000BZI/
Accession                                                                        ENCSR000BZI
Assay name                                                                           RNA-seq
Assay title                                                                    polyA RNA-seq
Target of assay                                                                          NaN
Target gene symbol                                                                       NaN
Biosample summary                                           C57BL/6 liver embryo (14.5 days)
Biosample term name                                                                    liver
Description                                The libraries contained in this experiment com...
Lab                                                                    Thomas Gingeras, CSHL
Project                                                               

In [0]:
# given the most important target of assay 
# could then look into Assay and Target of assay 

# applying jaccard similarity
def jaccard_similarity(i,j): 
    intersection = set(i).intersection(set(j))
    union = set(i).union(set(j))
    return len(intersection)/len(union)
  
def find_max_similarity(list_a): 
  max_num = 0
  ids=[]
  list_a = list(map(float, list_a))
  for i in range(len(list_a)): 
    if list_a[i]>max_num: 
      max_num=list_a[i]
    if list_a[i]==max_num:
      ids.append(i)
  return ids, max_num
      
sim_matrix = []
ids_matrix = []
def get_similarity_matrix(matrix): 
  sim_matrix=[]
  scores=[]
  for i in range(len(matrix)): 
    list_a=[]
    if i>0:
      list_a.append("0"*i)
    for j in range(i,len(matrix)):
      if i==j: 
        list_a.append(-1)
      else:
        list_a.append(jaccard_similarity(matrix.loc[i], matrix.loc[j]))
    ids, max_num = find_max_similarity(list_a)
    ids_matrix.append(ids)
    scores.append({'similarity': max_num})
    sim_matrix.append(list_a)
  return sim_matrix, ids_matrix, scores

In [0]:
# get a small subset of the data to use 
to_test = relevant_data[:500]
#sim_matrix, ids_matrix, scores = get_similarity_matrix(to_test)

In [0]:
def make_new_df(to_test): 
  sim_matrix, ids_matrix, scores = get_similarity_matrix(to_test)
  to_show=[]
  x=0
  for i in ids_matrix: 
    list_of_datasets=[]
    for j in range(len(i)): 
      list_of_datasets.append(ids_of_data.loc[i[j]])
    to_show.append({'datasets':list_of_datasets})
    x+=1
  temp_col = pd.DataFrame(data=to_show)
  other_temp = pd.DataFrame(data=scores)
  to_test['ids']=temp_col
  to_test['similarity']=other_temp
  return to_test

In [14]:
updated_to_test = make_new_df(to_test)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [0]:
# TO SAVE FILES 
sim_matrix.to_csv("similarity_matrix.csv", index=False)
ids_matrix.to_csv("ids_matrix.csv", index=False)
scores.to_csv("max_scores.csv", index=False)

In [15]:
# UI for getting similarity between datasets 
# Incorporate bokeh server into this 
# make this work for various tickers 
# creating a bokeh app for recommendation system 

from bokeh.io import curdoc, output_notebook, show, push_notebook
from bokeh.layouts import row, column, widgetbox, layout
from bokeh.models import ColumnDataSource, Div, ColorBar, LinearColorMapper 
from bokeh.models.widgets import PreText, Slider,Button, Select, RangeSlider, TableColumn, DataTable
from ipywidgets import interact
from bokeh.models.callbacks import CustomJS
from bokeh.plotting import  output_file, figure
import io 
import base64

output_notebook()

# fixing messy column names
updated_to_test.columns = updated_to_test.columns.str.strip().str.lower().str.replace(" ", "_")

# TICKETS FOR TICKERS
TARGET_ASSAY_TICKETS = list(data['Target of assay'].unique())
CELLTYPE_TICKETS = list(data['Biosample term name'].unique())
ORGA_TICKETS = list(data['Organism'].unique())

source = ColumnDataSource(data=dict())

#TICKERS
ticker1 = Select(title="Select Target Assay:", value="Control", options=list(str(i) for i in TARGET_ASSAY_TICKETS))
ticker2 = Select(title="Select Celltype:", value="K562", options=list(str(i) for i in CELLTYPE_TICKETS))
ticker3 = Select(title="Select Organism:", value="Homo sapiens", options=list(str(i) for i in ORGA_TICKETS))

#x_axis = Select(title="X Axis", options=['accession', 'Organism'], value="accession")
#y_axis = Select(title="Y Axis", options=['similarity', 'Organism'], value="similarity")

slider = Slider(title="similarity_threshold", value=0.0, start=0.0, end=1.0, step=0.1)


### FUNCTIONS FOR FILE UPLOAD
def file_callback(attr, old, new): 
    print ('filename:', file_source.data['file_name'])
    raw_contents = file_source.data['file_contents'][0]
    # remove the prefix that JS adds  
    prefix, b64_contents = raw_contents.split(",", 1)
    file_contents = base64.b64decode(b64_contents)
    file_io = io.StringIO(bytes.decode(file_contents))
    df = pd.read_csv(file_io)
  

file_source = ColumnDataSource({'file_contents':[], 'file_name':[]})

def file_callback(attr,old,new):
    print ('filename:', file_source.data['file_name'])
    raw_contents = file_source.data['file_contents'][0]
    # remove the prefix that JS adds  
    prefix, b64_contents = raw_contents.split(",", 1)
    file_contents = base64.b64decode(b64_contents)
    file_io = io.StringIO(bytes.decode(file_contents))
    df = pd.read_csv(file_io)
    print ("file contents:")
    print (df)

file_source.on_change('data', file_callback)

button = Button(label="Upload", button_type="success")
button.callback = CustomJS(args=dict(file_source=file_source), code = """
function read_file(filename) {
    var reader = new FileReader();
    reader.onload = load_handler;
    reader.onerror = error_handler;
    // readAsDataURL represents the file's data as a base64 encoded string
    reader.readAsDataURL(filename);
}

function load_handler(event) {
    var b64string = event.target.result;
    file_source.data = {'file_contents' : [b64string], 'file_name':[input.files[0].name]};
    file_source.trigger("change");
}

function error_handler(evt) {
    if(evt.target.error.name == "NotReadableError") {
        alert("Can't read file!");
    }
}

var input = document.createElement('input');
input.setAttribute('type', 'file');
input.onchange = function(){
    if (window.FileReader) {
        read_file(input.files[0]);
    } else {
        alert('FileReader is not supported in this browser');
    }
}
input.click();
""")


def selected(): 
  target_assay = ticker1.value
  celltype =ticker2.value
  org = ticker3.value 
  x_name = x_axis.value
  y_name = y_axis.value
  selected = updated_to_test[(updated_to_test['similarity'] >= slider.value) & (updated_to_test['target_of_assay'] == target_assay) & (updated_to_test['biosample_term_name'] == celltype) & (updated_to_test['organism'] == org)]
  return selected, x_name, y_name
  
def update(): 
    df, x_name, y_name = selected()
    source.data = {
        'accession': df.accession,
        'similar': df.ids,
        'organism':df.organism,
        'biosample_term_name':df.biosample_term_name
    }
    
TOOLTIPS= [ 
  ("Accession", "@accession"),
   ("Similarity", "@similarity"), 
    ("Organism", "@organism"), 
    ("Cell Type", "@biosample_term_name"), 
    
]

  

controls = [ticker1, ticker2, ticker3] #x_axis, y_axis]
for control in controls: 
  control.on_change('value', lambda attrname, old, new:update())
slider.on_change('value', lambda attrname, old, new:update()) 
  
columns=[
   TableColumn(field="accession", title="Original IDs", width=50),
   TableColumn(field="similar", title="Dataset IDs")
]

inputs=column(*controls, width=100, height=300)
inputs.sizing_mode="fixed"
data_table = DataTable(source=source, columns=columns, width=1500)

l = layout([slider, button], 
           [inputs],
           [data_table])
          

curdoc().add_root(l)
curdoc().title="Recommendation Systeem"

update()
show(l, notebook_handle=True)


Loading BokehJS ...

You are generating standalone HTML/JS output, but trying to use real Python
callbacks (i.e. with on_change or on_event). This combination cannot work.

Only JavaScript callbacks may be used with standalone output. For more
information on JavaScript callbacks with Bokeh, see:

    http://bokeh.pydata.org/en/latest/docs/user_guide/interaction/callbacks.html

Alternatively, to use real Python callbacks, a Bokeh server application may
be used. For more information on building and running Bokeh applications, see:

    http://bokeh.pydata.org/en/latest/docs/user_guide/server.html



In [16]:

output_notebook()
source = ColumnDataSource(data=dict(to_test))
color_mapper = LinearColorMapper(palette="RdBu11", low=0.0, high=1.0)
color_bar = ColorBar(color_mapper=color_mapper, label_standoff=12, border_line_color=None, location=(0,0))

print(to_test.columns)
pic = figure(x_range=to_test['accession'].unique(), plot_height=500, plot_width=700, title="", toolbar_location=None, tooltips=TOOLTIPS, sizing_mode="scale_both")
pic.circle(x="accession", y="similarity", source=source, size=7, line_color=None)

TOOLTIPS= [ 
  ("Accession", "@accession"),
   ("Similarity", "@similarity"), 
    ("Organism", "@organism"), 
    ("Cell Type", "@biosample_term_name"),
    ("Target of Assay", "@target_of_assay")
]

show(pic)

# EACH COLUMN REPRESENTS THE ACCESSION NUMBER OF A GIVEN DATASET AND EACH POINT REPRESENTS A DATASET 


Loading BokehJS ...

Index(['accession', 'assay_name', 'assay_title', 'biosample_term_name',
       'target_of_assay', 'organism', 'biosample_treatment', 'ids',
       'similarity'],
      dtype='object')
